Join the data from Part 1 with the data from Part 2 to create a new dataframe.

In [92]:
#Getting necessary imports
import pandas as pd

Getting part 1 data, **citybikes_vancouver.csv**

In [93]:
#load city_bikes_vancouver.csv
cb_vancouver = pd.read_csv('../data/citybikes_vancouver.csv')

Getting part 2 data

1. Foursquare
2. Yelp

In [94]:
# Foursquare
#load cb_fsq_vancouver.csv
cb_fsq_vancouver = pd.read_csv('../data/cb_fsq_vancouver.csv')
#load cb_fsq_vancouver_tips
cb_fsq_vancouver_tips = pd.read_csv('../data/cb_fsq_tips_vancouver.csv')

In [95]:
#get the count of fsq_id in tips
tips_count = cb_fsq_vancouver_tips.groupby('fsq_id').size().reset_index(name='review_count')

#merge cb_fsq_vancouver and tips_count
cb_fsq_vancouver = pd.merge(cb_fsq_vancouver, tips_count, on='fsq_id', how='left')

In [96]:
#round lat and long to 3 decimal places
cb_fsq_vancouver['lat'] = cb_fsq_vancouver['lat'].round(3)
cb_fsq_vancouver['long'] = cb_fsq_vancouver['long'].round(3)

cb_fsq_vancouver.head()

,fsq_id,station,category,name,address,lat,long,site,cb_id,review_count
0,4fd3a01be4b0540d9cb027ea,10th & Cambie,Café,Moii Cafe,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",49.263,-123.115,Foursquare,cb_0,20.0
1,4bc290d674a9a593788ed3f6,10th & Cambie,Banking and Finance,Vancity Savings Credit Union,"501 10th Ave W, Vancouver BC V5Z 1K9",49.262,-123.115,Foursquare,cb_0,4.0
2,4b1db335f964a520211424e3,10th & Cambie,Café,Marulilu Cafe,"451W Broadway W (at Cambie St), Vancouver BC V...",49.263,-123.114,Foursquare,cb_0,30.0
3,4db0b65d4df03036e8c0960c,10th & Cambie,Hair Salon,Space Salon,"530W Broadway W (btw Ash & Cambie), Vancouver ...",49.263,-123.116,Foursquare,cb_0,12.0
4,4ab04bfaf964a520ed6620e3,10th & Cambie,Dim Sum Restaurant,Peaceful Restaurant,"532 Broadway W (btwn Cambie & Ash), Vancouver ...",49.263,-123.116,Foursquare,cb_0,10.0


In [97]:
#get postal codes from address
cb_fsq_vancouver['postal_code'] = cb_fsq_vancouver['address'].str.split(',').str[-1].str.strip()
#remove 'Vancouver BC' from address
cb_fsq_vancouver['postal_code'] = cb_fsq_vancouver['postal_code'].str.replace('Vancouver BC', '')
#remove space
cb_fsq_vancouver['postal_code'] = cb_fsq_vancouver['postal_code'].str.replace(' ','')

cb_fsq_vancouver.head()

,fsq_id,station,category,name,address,lat,long,site,cb_id,review_count,postal_code
0,4fd3a01be4b0540d9cb027ea,10th & Cambie,Café,Moii Cafe,"2559 Cambie St (btwn W Broadway & W 10th Ave),...",49.263,-123.115,Foursquare,cb_0,20.0,V5Z3Y6
1,4bc290d674a9a593788ed3f6,10th & Cambie,Banking and Finance,Vancity Savings Credit Union,"501 10th Ave W, Vancouver BC V5Z 1K9",49.262,-123.115,Foursquare,cb_0,4.0,V5Z1K9
2,4b1db335f964a520211424e3,10th & Cambie,Café,Marulilu Cafe,"451W Broadway W (at Cambie St), Vancouver BC V...",49.263,-123.114,Foursquare,cb_0,30.0,V5Y1R4
3,4db0b65d4df03036e8c0960c,10th & Cambie,Hair Salon,Space Salon,"530W Broadway W (btw Ash & Cambie), Vancouver ...",49.263,-123.116,Foursquare,cb_0,12.0,V5T1X5
4,4ab04bfaf964a520ed6620e3,10th & Cambie,Dim Sum Restaurant,Peaceful Restaurant,"532 Broadway W (btwn Cambie & Ash), Vancouver ...",49.263,-123.116,Foursquare,cb_0,10.0,V5Z1E9


In [98]:
#Yelp
#load cb_yelp_vancouver.csv
cb_yelp_vancouver = pd.read_csv('../data/cb_yelp_vancouver.csv')

In [99]:
#round the latitude and longitude to 2 decimal places
cb_yelp_vancouver['lat'] = cb_yelp_vancouver['lat'].round(3)
cb_yelp_vancouver['long'] = cb_yelp_vancouver['long'].round(3)
#drop square brackets and single quotes from the address column
cb_yelp_vancouver['address'] = cb_yelp_vancouver['address'].str.replace('[','').str.replace(']','').str.replace("'",'')

cb_yelp_vancouver.head()


C:\Users\milan\AppData\Local\Temp\ipykernel_2384\2726789357.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cb_yelp_vancouver['address'] = cb_yelp_vancouver['address'].str.replace('[','').str.replace(']','').str.replace("'",'')


,title,name,address,lat,long,rating,review_count,site,cb_id
0,Mexican,La Taqueria Pinche Taco Shop,"2450 Yukon Street, Vancouver, BC V5Z 3V6, Canada",49.264,-123.113,4.0,680,Yelp,cb_0
1,Noodles,Peaceful Restaurant,"110-532 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.116,3.5,620,Yelp,cb_0
2,Cafes,Marulilu Cafe,"451 W Broadway, Vancouver, BC V5Y 1R4, Canada",49.263,-123.114,4.0,284,Yelp,cb_0
3,Japanese,Saku,"548 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.117,4.5,202,Yelp,cb_0
4,Soup,Hokkaido Ramen Santouka,"558 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.117,4.0,221,Yelp,cb_0


In [100]:
#from address column get the postal code
cb_yelp_vancouver['postal_code'] = cb_yelp_vancouver['address'].str.split(',').str[-2].str.strip()
#remove 'BC' from postal_code column
cb_yelp_vancouver['postal_code'] = cb_yelp_vancouver['postal_code'].str.replace('BC','')
#remove space from postal_code column
cb_yelp_vancouver['postal_code'] = cb_yelp_vancouver['postal_code'].str.replace(' ','')

cb_yelp_vancouver.head()


,title,name,address,lat,long,rating,review_count,site,cb_id,postal_code
0,Mexican,La Taqueria Pinche Taco Shop,"2450 Yukon Street, Vancouver, BC V5Z 3V6, Canada",49.264,-123.113,4.0,680,Yelp,cb_0,V5Z3V6
1,Noodles,Peaceful Restaurant,"110-532 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.116,3.5,620,Yelp,cb_0,V5Z1E9
2,Cafes,Marulilu Cafe,"451 W Broadway, Vancouver, BC V5Y 1R4, Canada",49.263,-123.114,4.0,284,Yelp,cb_0,V5Y1R4
3,Japanese,Saku,"548 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.117,4.5,202,Yelp,cb_0,V5Z1E9
4,Soup,Hokkaido Ramen Santouka,"558 W Broadway, Vancouver, BC V5Z 1E9, Canada",49.263,-123.117,4.0,221,Yelp,cb_0,V5Z1E9


In [ ]:
# join cb_fsq_vancouver and cb_yelp_vancouver through cb_id and postal_code
cb_fsq_yelp_vancouver = pd.merge(cb_fsq_vancouver, cb_yelp_vancouver, on=['cb_id','postal_code'], how='outer')

#fill Nan values with 'Not Available'
cb_fsq_yelp_vancouver = cb_fsq_yelp_vancouver.fillna('unavilable')

cb_fsq_yelp_vancouver.head()

In [ ]:
#drop name from cb_vancouver
cb_vancouver = cb_vancouver.drop('name', axis=1)

#join cb_fsq_yelp_vancouver and cb_vancouver through cb_id
main_df = pd.merge(cb_fsq_yelp_vancouver, cb_vancouver, on='cb_id', how='outer')

main_df.head()

Provide a visualization that you used as part of your EDA process. Explain the initial pattern or relationship you discoved through this visualization. 

# Database

Put all your results in an SQLite3 database (remember, SQLite stores its databases as files in your local machine - make sure to create your database in your project's data/ directory!)

Look at the data before and after the join to validate your data.